In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import plotly.express as px

<h3> Lance une requête sur la page web : [Chuck Norris facts]

(https://chucknorrisfacts.net/facts.php?page=1)

In [2]:
url = 'https://chucknorrisfacts.net/facts.php?page=1'
navigator = 'Chrome/96.0.4664.93'
html = requests.get(url, headers={'User-Agent': navigator})
soup = bs(html.text, 'html.parser')
html

<Response [200]>

<h3> Observe attentivement l'URL de la page 1. <br /> Essaye de passer à la page 2.<br />  Dans un notebook python, essaye de générer automatiquement les URL des 5 premières pages (avec une boucle par exemple ?)

In [3]:
#pages 2 à 5 

list_url = []

for page in range(2,6) :
  link = 'https://chucknorrisfacts.net/facts.php?page='+str(page)
  list_url.append(link)

list_url[1]

'https://chucknorrisfacts.net/facts.php?page=3'

In [4]:
list_url

['https://chucknorrisfacts.net/facts.php?page=2',
 'https://chucknorrisfacts.net/facts.php?page=3',
 'https://chucknorrisfacts.net/facts.php?page=4',
 'https://chucknorrisfacts.net/facts.php?page=5']

<h3> Utilise BeautifulSoup pour scraper les facts et les notes des 5 premières pages, grâce aux URL que tu as générées. <br /> (Nous pourrions faire les 100 ou 200 pages suivantes, mais ca risque d'entrainer une charge inutile pour ceux qui administrent ce serveur.) <br /> Le résultat doit être dans un dictionnaire.

In [5]:
# créer une liste vide pour remplir des fun facts Chuck Norris
fun_facts = []
rate_facts =[]

#créer une liste vide qui contiendra les pages url à scrapper
list_url2 = []

#création de la première boucle pour ajouter les url par page du site
for page in range(1,6) :
  link = 'https://chucknorrisfacts.net/facts.php?page='+str(page)
  list_url2.append(link)

#création de la boucle sous la premiere pour ouvrir les pages contenues dans la liste list_url2 + récupération des fun fact et des notes
  for i in list_url2:
    url = i
    navigator = 'Chrome/96.0.4664.93'
    html = requests.get(url, headers={'User-Agent': navigator})
    soup = bs(html.text, 'html.parser')
    joke = soup.find(id='content')
    facts = joke.find_all('p')
    rate = joke.find_all('span', {'class':"out5Class"})

#création de la boucle pour ajouter le sfun facts de chaque page dans la liste fun_facts & idem pour les notes   
  for x in range(len(facts)):
    sentence = facts[x].get_text()
    fun_facts.append(sentence)
  
  for r in range(len(rate)):
    rate_fact_chuck = rate[r].get_text()
    rate_facts.append(rate_fact_chuck)
    

print(len(fun_facts))
print(len(rate_facts))


100
100


In [6]:
#ajout dans un dictionnaire

keys = fun_facts
values = rate_facts

chuck_dict = dict(zip(keys, values))
len(chuck_dict)






100

<h3> Crée un DataFrame contenant ces données, il doit donc posséder 2 colonnes et 100 lignes.

In [7]:
df_chuck = pd.Series(chuck_dict)
df_chuck = pd.DataFrame(df_chuck).reset_index()
df_chuck.rename({'index' : 'Joke', 0 : "Rate"}, axis =1, inplace=True)
df_chuck.sort_values('Rate', ascending=False, inplace=True)
df_chuck

,Joke,Rate
20,Chuck Norris can kill two stones with one bird.,4.09
75,"Chuck actually died four years ago, but the Gr...",4.08
74,"When Chuck Norris was in middle school, his En...",4.08
3,Chuck Norris' calendar goes straight from Marc...,4.08
34,"There is no theory of evolution, just a list o...",4.08
...,...,...
67,Chuck Norris sheds his skin twice a year.,2.55
40,"Chuck Norris doesn�t wash his clothes, he dise...",2.37
30,Chuck Norris drives an ice cream truck covered...,2.13
25,"Chuck Norris is ten feet tall, weighs two-tons...",1.83


<h3> Crée une visualisation de la distribution des notes.

In [8]:
# Convertion de la colonne Rate qui est en str, en float
df_chuck = df_chuck.astype({'Rate': float})

In [9]:
# Regroupement des notes en 5 groupes <1,<2,<3,<4,<5

score = []
sum_rate = []

for i in range (1,6):
  index = "Inférieur à "+str(i)+ "/5"
  score.append(index)
  rate = df_chuck[(df_chuck['Rate'] >=i) & (df_chuck['Rate'] < i+1)].count().sum()
  sum_rate.append(rate)

print(score)
print(sum_rate)

['Inférieur à 1/5', 'Inférieur à 2/5', 'Inférieur à 3/5', 'Inférieur à 4/5', 'Inférieur à 5/5']
[4, 16, 134, 46, 0]


In [10]:
#ajout dans un dictionnaire

keys = score
values = sum_rate

score_dict = dict(zip(keys, values))
score_dict


{'Inférieur à 1/5': 4,
 'Inférieur à 2/5': 16,
 'Inférieur à 3/5': 134,
 'Inférieur à 4/5': 46,
 'Inférieur à 5/5': 0}

In [11]:
#create df
df_rate = pd.Series(score_dict)
df_rate = pd.DataFrame(df_rate).reset_index()
df_rate.rename({'index' : 'Score', 0 : "Number_of_rate"}, axis =1, inplace=True)

df_rate

,Score,Number_of_rate
0,Inférieur à 1/5,4
1,Inférieur à 2/5,16
2,Inférieur à 3/5,134
3,Inférieur à 4/5,46
4,Inférieur à 5/5,0


In [12]:
#create graph

fig = px.bar(data_frame = df_rate,  x="Score", y= "Number_of_rate",
             color="Score", color_discrete_sequence=px.colors.qualitative.Pastel )
           

fig.update_layout(width=1000, height=600, title='Count Score by joke of Chuck Norris', template='plotly_dark')

fig.show()

